## Transfer Learning by Feature Extraction, Average Ensemble Network 7 for Visualisation 10 fold Seed 42 and seed 7

Repository: https://github.com/ZainK-hub/satbinclass

This notebook uses the versions listed below of the following packages:
* Python: 3.7.4
* PANDAS: 0.25.1
* Numpy: 1.17.2
* Matplotlib: 3.1.1
* Scikit-learn: 0.21.3
* Scipy: 1.3.1
* Tensorflow-GPU 1.14.0
* Tensorflow 1.14.0
* CUDA toolkit: 10.1.168
* CUDNN: 7.6.0
* Keras-GPU: 2.2.4
* Keras-preprocessing: 1.1.0

In [1]:
#Seed random generators to ensure reproducible results
from numpy.random import seed
#seed(7)
from tensorflow import set_random_seed 
#set_random_seed(8)
import efficientnet.keras as efn
from efficientnet.keras import center_crop_and_resize #, preprocess_input

import time
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils, to_categorical, plot_model
from keras.applications import imagenet_utils, resnet50, resnet
from keras.applications import resnext, inception_resnet_v2
from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
import matplotlib.pyplot as plt
from keras.backend import clear_session
from datetime import datetime

/home/learner/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/learner/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/learner/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/learner/anaconda3/envs/tfgpu/lib/python3

In [2]:
import tensorflow  as tf
from tensorflow.python.client import device_lib
#device_lib.list_local_devices()


## Introduction

This notebook contains the code for image classification using Transfer Learning by Feature Extraction, Average Ensemble Network 7 (AvgEnsNet 7) with 10-fold cross validation. A *random_state* or *seed* value of both 42 and 7 in turn are used in determining the data splits. This note book generates the predictions for the best performing classifier to see where the classifier struggles to classify the samples correctly.

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, \
    confusion_matrix, make_scorer
import pandas as pd

## Data preprocessing

The data is read in and preprocessed. Each model's own preprocessing is used before using it for Feature Extraction to generate features. Many Machine Learning algorithms may perform badly if the features do not appear to be standard normally distributed data i.e. a Gaussian with unit variance and mean of zero. Hence the features are standardised/normalised before being fed into the Logistic Regression model.

*StratifiedShuffleSplit* is used to ensure that the data is split and shuffled in such a way that the percentage of the categories of the data relative to the original dataset is maintained. A test dataset is made and separated from the dataset that will be used to train the model so as to not train on the data used to quantify the model performance since cross-validation is used.

In [4]:
X = np.load('X.npy')
y = np.load('y.npy')
#One hot encoding
one_hot_y = to_categorical(y)
#y = one_hot_y
print("X: ", X.shape)

X:  (224, 224, 6, 1440)


In [5]:
print(y.shape)
y = np.ravel(y)
y.shape

(1440, 1)


(1440,)

In [6]:
img_res = X.shape[0]
blobs = X.shape[3]
X = X.T
print(X.shape)

(1440, 6, 224, 224)


In [7]:
#Scale the disp, ph and coh bands to 0 to 255
def scaleBandData(X, img_res):
    blobs = X.shape[0]
    bands = X.shape[1]
    print('band input shape', X.shape)
    X = X.T
    print('scale band data shape', X.shape)
    X = X.reshape(X.shape[0]*X.shape[1]*X.shape[2], X.shape[3]).T
    print('scale band data reshaped', X.shape)
    scaler = MinMaxScaler(feature_range=(0,255), copy=False)
    scaler = scaler.fit(X)
    X = scaler.transform(X)
    X = X.reshape(blobs, bands, img_res, img_res)
    print('band output shape', X.shape)
    return X
print('max, min', np.max(X[:,0:3,:,:]), np.min(X[:,0:3,:,:]))
X[:,0:3,:,:] = scaleBandData(X[:,0:3,:,:], img_res)
print('max, min', np.max(X[:,0:3,:,:]), np.min(X[:,0:3,:,:]))

max, min 3.1391279697418213 -3.1403708457946777
band input shape (1440, 3, 224, 224)
scale band data shape (224, 224, 3, 1440)
scale band data reshaped (1440, 150528)
band output shape (1440, 3, 224, 224)
max, min 255.00000000000006 0.0


In [8]:
print(X.shape)
print('max, min', np.max(X), np.min(X))

(1440, 6, 224, 224)
max, min 255.00000000000006 0.0


In [9]:
#Reshape to Keras desired shape
X = X.reshape(X.shape[0], X.shape[2], X.shape[3], X.shape[1])
print(X.shape)

(1440, 224, 224, 6)


## Classification

The classifiers are implemented below.

In [10]:
#Setup single models
def singleModel(X_features_train_val, X_features_test, y_train, y_val, y_test, train_index, val_index, 
                single_model_name_str, results_arr, clf):
    
    print(single_model_name_str)
    X_train = X_features_train_val[train_index].copy()
    X_val = X_features_train_val[val_index].copy()
    X_test = X_features_test.copy()
    print('Train: ', X_train.shape, 'Validation:', X_val.shape, 'Test:', X_test.shape)         

    clf.fit(X_train, y_train)
    preds = clf.predict_proba(X_val)

    val_acc = round(accuracy_score(y_val, np.round(np.argmax(preds, axis=1)))*100, 2)
    print('Validation score of ' + single_model_name_str + ' model: ', val_acc, '%')
    
    print('Grid search has completed. \n Mean cross-validated score of the best estimator is:', clf.best_score_*100)
    print('The best parameters are: ', clf.best_params_, '\n')
    results_arr.append(clf.best_params_)
    
    results_arr.append(val_acc)      
    test_preds = clf.predict_proba(X_test)
    test_acc = round(accuracy_score(y_test.copy(), np.round(np.argmax(test_preds,axis=1)))*100, 2)
    print('Testing Accuracy on totally unseen data of ' + single_model_name_str + ' model: ', test_acc, '%')
    results_arr.append(test_acc)     
    results_arr = evaluator(np.round(np.argmax(test_preds,axis=1)), y_test.copy(), results_arr)
    del X_train, X_val, X_test, y_train
    
    return results_arr, preds, test_preds

#Avg./Real Ensembler, input array of mean preds
def realEnsembler(y_val, y_test, ensemble_model_name_str, results_arr, preds, test_preds):
    val_acc = round(accuracy_score(y_val, np.round(np.argmax(preds, axis=1)))*100, 2)
    print('Validation score of Ensemble: ' + ensemble_model_name_str + ' model: ', val_acc, '%')
    
    results_arr.append("Ensemble")
    
    results_arr.append(val_acc)      
    test_acc = round(accuracy_score(y_test.copy(), np.round(np.argmax(test_preds,axis=1)))*100, 2)
    print('Testing Accuracy on totally unseen data of ' + ensemble_model_name_str + ' model: ', test_acc, '%')
    results_arr.append(test_acc)     
    results_arr = evaluator(np.round(np.argmax(test_preds,axis=1)), y_test.copy(), results_arr)    
    return results_arr

#Evaluate performance
def evaluator(test_preds, y_test, results_arr):
    f1_sco = round(f1_score(y_test, np.round(test_preds), average='macro'), 2)
    precision = round(precision_score(y_test, np.round(test_preds), average='macro'), 2)
    recall = round(recall_score(y_test, np.round(test_preds), average='macro'), 2)
    print('F1 score is: ', f1_sco)
    print('Recall score is: ', precision)
    print('Precision score is: ', recall)

    confu = confusion_matrix(y_test, test_preds)
    print('Confusion matrix: \n', confu)
    print('From the confusion matrix above there are ' + str(confu[0, 0]) + ' true negatives and ' + str(confu[0, 1]) 
          + ' false positives.') 
    print('There are ' + str(confu[1, 0]) + ' false negatives and ' + str(confu[1, 1]) + ' true positives. \n')
    performance = np.array((f1_sco, precision, recall, confu[0, 0], confu[0, 1], confu[1, 0], confu[1, 1]))
    for i in range(len(performance)):
        results_arr.append(performance[i])
    return results_arr
    
#Ensemble models
def ensembler(X_features_train_val, X_features_test):
    X_ensemble_features_train_val = combiner(X_features_train_val)
    del X_features_train_val
    X_ensemble_features_test = combiner(X_features_test)
    del X_features_test
    return X_ensemble_features_train_val, X_ensemble_features_test

def combiner(features):
    for i in range(len(features)):
        if i==0:
            combined_features=features[i]
        if i !=0:
            combined_features=np.hstack((combined_features,features[i][:,1:]))
    return combined_features

In [11]:
#Function to train model 
def trainModel(single_model_name, ensemble_names, results_data, folds, seed_value, X_1_features_train_val, X_1_features_test, y_train_val, y_test, X_2_features_train_val, X_2_features_test,
              X_3_features_train_val, X_3_features_test,):
    sss = StratifiedShuffleSplit(n_splits=folds, test_size=0.2, random_state=seed_value)
    sss.get_n_splits(X_1_features_train_val, y_train_val)      

    results_arr = []
    test_preds_arr = np.array(())

    param_grid = dict(logisticregression__C=[0.001, 0.0005, 0.0001,])
    #refit=True means at end it will train the best parameters on all the data and tune the hyperparameters to it
    pipe = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=1000))
    clf = []
    
    for i in range(len(single_model_name)):
        clf.append(GridSearchCV(pipe, param_grid=param_grid, cv=sss, scoring=make_scorer(accuracy_score), refit=True))
    
    counter = 0
    for train_index, val_index in sss.split(X_1_features_train_val, y_train_val):
        #Test to see if there any overlapped indices
        print('Overlap',set(train_index) & set (val_index))
        print('\n' + 'Counter: ', counter,)
        print(datetime.now())
        
        results_arr.append(counter)
        
        y_train = y_train_val[train_index].copy()
        y_val = y_train_val[val_index].copy()
        y_test = y_test.copy()
                
        # Model 1 
        results_arr, resnet50_preds, resnet50_test_preds = singleModel(X_1_features_train_val, X_1_features_test, y_train.copy(), 
                            y_val.copy(), y_test.copy(), train_index, val_index, single_model_name[0], results_arr, clf[0])

        # Model 2
        results_arr, resnext50_preds, resnext50_test_preds = singleModel(X_2_features_train_val, X_2_features_test, y_train.copy(), 
                            y_val.copy(), y_test.copy(), train_index, val_index, single_model_name[1], results_arr, clf[1])
        
        # Model 3
        results_arr, effnetb4_preds, effnetb4_test_preds = singleModel(X_3_features_train_val, X_3_features_test, y_train.copy(), 
                            y_val.copy(), y_test.copy(), train_index, val_index, single_model_name[2], results_arr, clf[2])
                      
        #AvgEnsNet 7: ResNet50 + ResNeXt50 + EfficientNetB4
        real_ens_preds = [resnet50_preds, resnext50_preds, effnetb4_preds]
        real_ens_preds = np.mean(real_ens_preds, axis=0)
        real_ens_test_preds = [resnet50_test_preds, resnext50_test_preds, effnetb4_test_preds]
        real_ens_test_preds = np.mean(real_ens_test_preds, axis=0)
        results_arr = realEnsembler(y_val.copy(), y_test.copy(), ensemble_names[0], results_arr, real_ens_preds, 
                                    real_ens_test_preds)
        if counter == 0:
            test_preds_arr = real_ens_test_preds
        else:
            test_preds_arr = np.hstack((test_preds_arr, real_ens_test_preds))
        del real_ens_preds, real_ens_test_preds

        #Append to data frame    
        temp_df2 = pd.DataFrame([results_arr], columns=results_data.columns) 
        results_data = results_data.append(temp_df2, ignore_index=True, sort=False)
        print('results_data \n\n', results_data)
        results_arr.clear()
                
        clear_session()
        counter = counter + 1
        del y_train, y_val, 
        
    print(str(folds) + '-fold cross validation has completed. \n Mean scores are :')
    print(results_data.mean(axis=0))
    print('test_preds_arr final', test_preds_arr.shape)
    
    return results_data, test_preds_arr

In [12]:
#Extract Features
def featureExtractor(model, X_band, X_rgb, batch_Size):
    #float32 gives faster speed and negligible loss of precision
    X_band = X_band.astype('float32')
    X_rgb = X_rgb.astype('float32')
    
    #Transfer Learning, feature extraction per 3 band image 
    features_1 = model.predict(X_band, batch_size=batch_Size)
    features_2 = model.predict(X_rgb, batch_size=batch_Size)
    del X_band, X_rgb

    features_1 = features_1.reshape((features_1.shape[0], features_1.shape[1]*features_1.shape[2]*features_1.shape[3]))
    features_2 = features_2.reshape((features_2.shape[0], features_2.shape[1]*features_2.shape[2]*features_2.shape[3]))
    X_feat = np.hstack((features_1,features_2))
    del features_1, features_2
    print("Features shape: ", X_feat.shape)
    return X_feat

In [13]:
def initiate(single_model_name, ensemble_names, results, preds_df, folds, seed_value, X, y):
    start = time.time()
    print('X, y:', X.shape, y.shape)
    print(datetime.now())

    #Split the data to get a hold out test set
    sss_initial = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed_value)
    num_splits = sss_initial.get_n_splits(X, y)

    for train_val_index, test_index in sss_initial.split(X, y):
        #print('Train-Validation: ', train_val_index, 'Testing', test_index)

        X_train_val = X[train_val_index].copy()
        y_train_val = y[train_val_index].copy()

        X_test = X[test_index].copy()
        y_test = y[test_index].copy()
        
        test_index_arr = np.array(test_index.tolist())
        del X, y,

        #Using pretrained Deep Learning model without retraining it
        img_h = img_res
        img_w = img_res
        batch_Size = 64

        #Image preprocessing ResNet50
        X_resnet50_band_train_val = resnet.preprocess_input(X_train_val[:,:,:,0:3].copy()) #Disp, phase and coherence treated as one 3 band image
        X_resnet50_rgb_train_val = resnet.preprocess_input(X_train_val[:,:,:,3:6].copy()) #RGB Sentinel image 
        print('X_resnet50_band_train_val processed: max, min', np.max(X_resnet50_band_train_val), np.min(X_resnet50_band_train_val))
        print('X_resnet50_rgb_train_val processed: max, min', np.max(X_resnet50_rgb_train_val), np.min(X_resnet50_rgb_train_val))
        X_resnet50_band_test = resnet.preprocess_input(X_test[:,:,:,0:3].copy()) #Disp, phase and coherence treated as one 3 band image
        X_resnet50_rgb_test = resnet.preprocess_input(X_test[:,:,:,3:6].copy()) #RGB Sentinel image 
        print('X_resnet50_band_test processed: max, min', np.max(X_resnet50_band_test), np.min(X_resnet50_band_test))
        print('X_resnet50_rgb_test processed: max, min', np.max(X_resnet50_rgb_test), np.min(X_resnet50_rgb_test))

        #Generating Features of ResNetV1 Models
        print("\n Generating features of ResNet models \n")
        resnet50_model = resnet.ResNet50(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
        X_resnet50_features_train_val = featureExtractor(resnet50_model, X_resnet50_band_train_val.copy(), X_resnet50_rgb_train_val.copy(), 
                                                         batch_Size)
        X_resnet50_features_test = featureExtractor(resnet50_model, X_resnet50_band_test.copy(), X_resnet50_rgb_test.copy(), 
                                                    batch_Size)
        del resnet50_model, X_resnet50_band_train_val, X_resnet50_rgb_train_val, X_resnet50_band_test, X_resnet50_rgb_test,
        clear_session()


        #Image preprocessing ResNeXt50
        X_resneXt50_band_train_val = resnext.preprocess_input(X_train_val[:,:,:,0:3].copy()) #Disp, phase and coherence treated as one 3 band image
        X_resneXt50_rgb_train_val = resnext.preprocess_input(X_train_val[:,:,:,3:6].copy()) #RGB Sentinel image 
        print('X_resneXt50_band_train_val processed: max, min', np.max(X_resneXt50_band_train_val), np.min(X_resneXt50_band_train_val))
        print('X_resneXt50_rgb_train_val processed: max, min', np.max(X_resneXt50_rgb_train_val), np.min(X_resneXt50_rgb_train_val))
        X_resneXt50_band_test = resnext.preprocess_input(X_test[:,:,:,0:3].copy()) #Disp, phase and coherence treated as one 3 band image
        X_resneXt50_rgb_test = resnext.preprocess_input(X_test[:,:,:,3:6].copy()) #RGB Sentinel image 
        print('X_resneXt50_band_test processed: max, min', np.max(X_resneXt50_band_test), np.min(X_resneXt50_band_test))
        print('X_resneXt50_rgb_test processed: max, min', np.max(X_resneXt50_rgb_test), np.min(X_resneXt50_rgb_test))

        #Generating Features of ResNeXt50 Model
        print("\n Generating features of ResNeXt models \n")
        resneXt50_model = resnext.ResNeXt50(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
        X_resneXt50_features_train_val = featureExtractor(resneXt50_model, X_resneXt50_band_train_val, X_resneXt50_rgb_train_val, 
                                                          batch_Size)
        X_resneXt50_features_test = featureExtractor(resneXt50_model, X_resneXt50_band_test, X_resneXt50_rgb_test, batch_Size)
        del X_resneXt50_band_train_val, X_resneXt50_rgb_train_val, X_resneXt50_band_test, X_resneXt50_rgb_test, resneXt50_model
        clear_session()


        #Image preprocessing EfficientNet B4
        X_effnet_band_train_val = X_train_val[:,:,:,0:3].copy()
        X_effnet_rgb_train_val = X_train_val[:,:,:,3:6].copy()
        X_effnet_band_test = X_test[:,:,:,0:3].copy()
        X_effnet_rgb_test = X_test[:,:,:,3:6].copy()

        image_size = 224

        for i in range(X_train_val.shape[0]):
            X_effnet_band_train_val[i,:,:,:] = center_crop_and_resize(X_effnet_band_train_val[i,:,:,:], image_size=image_size) #Disp, phase and coherence treated as one 3 band image
            X_effnet_rgb_train_val[i,:,:,:] = center_crop_and_resize(X_effnet_rgb_train_val[i,:,:,:], image_size=image_size) #RGB Sentinel image 
            X_effnet_band_train_val[i,:,:,:] = efn.preprocess_input(X_effnet_band_train_val[i,:,:,:]) #Disp, phase and coherence treated as one 3 band image
            X_effnet_rgb_train_val[i,:,:,:] = efn.preprocess_input(X_effnet_rgb_train_val[i,:,:,:]) #RGB Sentinel image 

        for i in range(X_test.shape[0]):
            X_effnet_band_test[i,:,:,:] = center_crop_and_resize(X_effnet_band_test[i,:,:,:], image_size=image_size) #Disp, phase and coherence treated as one 3 band image
            X_effnet_rgb_test[i,:,:,:] = center_crop_and_resize(X_effnet_rgb_test[i,:,:,:], image_size=image_size) #RGB Sentinel image 
            X_effnet_band_test[i,:,:,:] = efn.preprocess_input(X_effnet_band_test[i,:,:,:]) #Disp, phase and coherence treated as one 3 band image
            X_effnet_rgb_test[i,:,:,:] = efn.preprocess_input(X_effnet_rgb_test[i,:,:,:]) #RGB Sentinel image 
        print('X_effnet_bands_train_val processed: max, min', np.max(X_effnet_band_train_val), np.min(X_effnet_band_train_val))
        print('X_effnet_rgb_train_val processed: max, min', np.max(X_effnet_rgb_train_val), np.min(X_effnet_rgb_train_val))
        print('X_effnet_bands_test processed: max, min', np.max(X_effnet_band_test), np.min(X_effnet_band_test))
        print('X_effnet_rgb_test processed: max, min', np.max(X_effnet_rgb_test), np.min(X_effnet_rgb_test))

        #Generating Features of EfficientNet Models
        print("\n Generating features of EfficientNet B4 model \n")
        effnetb4_model = efn.EfficientNetB4(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
        X_effnetb4_features_train_val = featureExtractor(effnetb4_model, X_effnet_band_train_val.copy(), X_effnet_rgb_train_val.copy(), 
                                                         batch_Size)
        X_effnetb4_features_test = featureExtractor(effnetb4_model, X_effnet_band_test.copy(), X_effnet_rgb_test.copy(), batch_Size)
        del effnetb4_model
        clear_session()

        results, test_preds = trainModel(single_model_name, ensemble_names, results, folds, seed_value, 
                            X_resnet50_features_train_val, X_resnet50_features_test, y_train_val, y_test, 
                            X_resneXt50_features_train_val, X_resneXt50_features_test, X_effnetb4_features_train_val,
                            X_effnetb4_features_test,)

        temp3 = np.vstack((test_index_arr, y_test, test_preds.T,))
        preds_df = pd.DataFrame(temp3.T, columns=preds_df.columns)
        del test_index_arr, test_preds, temp3
        
        del X_resnet50_features_train_val, X_resnet50_features_test, y_train_val,
        del X_resneXt50_features_train_val, X_resneXt50_features_test, X_effnetb4_features_train_val, X_effnetb4_features_test


    end = time.time()
    elapsed = end - start
    print('Total running time (h): %.2f hours' % (elapsed/3600.0))
    print(datetime.now())
    return results, preds_df

folds = 10
single_model_name = ['ResNet50', 'ResNeXt50', 'EfficientNetB4',]

for i in range(len(single_model_name)):
    print('Model ' + str(i + 1) + ': ' + single_model_name[i])

#Results column names
col_names = ['Counter',]
subnames = ['Best Param', 'Val Acc', 'Test Acc', 'F1 score', 'Precision', 'Recall', 'True Neg.', 'False Pos.', 'False Neg.', 'True Pos.',]
for i in range(len(single_model_name)):
    for j in range(len(subnames)):
        col_names.append(single_model_name[i] + ' ' + subnames[j])

ensemble_names = ['AvgEnsNet 7: ResNet50-ResNeXt50-EffNetB4',]

for i in range(len(ensemble_names)):
    for j in range(len(subnames)):
        col_names.append(ensemble_names[i] + ' ' + subnames[j])

results_data = pd.DataFrame(columns=col_names)

test_cols_0 = 'Test Preds 0'
test_cols_1 = 'Test Preds 1'
preds_cols = ['Test Index', 'Target']
for i in range(folds):
    preds_cols.append('Fold: ' + str(i + 1) + ' ' + test_cols_0)
    preds_cols.append('Fold: ' + str(i + 1) + ' ' + test_cols_1)

preds_data = pd.DataFrame(columns=preds_cols)

results_data, preds_data = initiate(single_model_name, ensemble_names, results_data, preds_data, folds, 42, X, y)    
preds_data.to_csv('./model/' + 'FE_and_AvgEnsNet7_' + '_' + 
               str(folds) + '_fold_crossval_Seed_42_Preds.csv', sep=',', index=False)

results_data, preds_data = initiate(single_model_name, ensemble_names, results_data, preds_data, folds, 7, X, y)
preds_data.to_csv('./model/' + 'FE_and_AvgEnsNet7_' + '_' + 
               str(folds) + '_fold_crossval_Seed_7_Preds.csv', sep=',', index=False)

results_copy  = results_data.copy()
temp_df2 = pd.DataFrame(results_data.mean(axis=0))
results_data = results_data.append(temp_df2.T, ignore_index=True, sort=False)
temp_df2 = pd.DataFrame(results_copy.var(axis=0))
results_data = results_data.append(temp_df2.T, ignore_index=True, sort=False)
temp_df2 = pd.DataFrame(results_copy.std(axis=0))
results_data = results_data.append(temp_df2.T, ignore_index=True, sort=False)


#Save results to CSV
results_data.to_csv('./model/' + 'FE_and_AvgEnsNet7_' + '_' + 
               str(folds) + '_fold_crossval_Seed_42_and_Seed_7_Results.csv', sep=',', index=False)



Model 1: ResNet50
Model 2: ResNeXt50
Model 3: EfficientNetB4
X, y: (1440, 224, 224, 6) (1440,)
2021-08-31 21:01:24.057257
X_resnet50_band_train_val processed: max, min 151.06100000000006 -123.68
X_resnet50_rgb_train_val processed: max, min 151.06100000000006 -123.68
X_resnet50_band_test processed: max, min 151.06100000000006 -123.68
X_resnet50_rgb_test processed: max, min 151.06100000000006 -123.68

 Generating features of ResNet models 








Features shape:  (1152, 200704)
Features shape:  (288, 200704)
X_resneXt50_band_train_val processed: max, min 2.640000000000001 -2.1179039301310043
X_resneXt50_rgb_train_val processed: max, min 2.640000000000001 -2.1179039301310043
X_resneXt50_band_test processed: max, min 2.640000000000001 -2.1179039301310043
X_resneXt50_rgb_test processed: max, min 2.640000000000001 -2.1179039301310043

 Generating features of ResNeXt models 

Features shape:  (1152, 200704)
Features shape:  (288, 200704)
X_effnet_bands_train_val processed: max, min 2.6400000

Train:  (921, 175616) Validation: (231, 175616) Test: (288, 175616)
Validation score of EfficientNetB4 model:  84.85 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 83.35135135135135
The best parameters are:  {'logisticregression__C': 0.0005} 

Testing Accuracy on totally unseen data of EfficientNetB4 model:  79.86 %
F1 score is:  0.8
Recall score is:  0.8
Precision score is:  0.8
Confusion matrix: 
 [[110  34]
 [ 24 120]]
From the confusion matrix above there are 110 true negatives and 34 false positives.
There are 24 false negatives and 120 true positives. 

Validation score of Ensemble: AvgEnsNet 7: ResNet50-ResNeXt50-EffNetB4 model:  87.88 %
Testing Accuracy on totally unseen data of AvgEnsNet 7: ResNet50-ResNeXt50-EffNetB4 model:  86.81 %
F1 score is:  0.87
Recall score is:  0.87
Precision score is:  0.87
Confusion matrix: 
 [[120  24]
 [ 14 130]]
From the confusion matrix above there are 120 true negatives and 24 false positives.
There are 14 f

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNet50 model:  84.42 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 81.83783783783784
The best parameters are:  {'logisticregression__C': 0.0001} 

Testing Accuracy on totally unseen data of ResNet50 model:  84.72 %
F1 score is:  0.85
Recall score is:  0.85
Precision score is:  0.85
Confusion matrix: 
 [[122  22]
 [ 22 122]]
From the confusion matrix above there are 122 true negatives and 22 false positives.
There are 22 false negatives and 122 true positives. 

ResNeXt50
Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  82.25 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 82.97297297297297
The best parameters are:  {'logisticregression__C': 0.001} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  80.21 %
F1 score is:  0.8
Recall score is:  0.8
Precision sc

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNet50 model:  84.42 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 83.89189189189189
The best parameters are:  {'logisticregression__C': 0.0001} 

Testing Accuracy on totally unseen data of ResNet50 model:  86.11 %
F1 score is:  0.86
Recall score is:  0.86
Precision score is:  0.86
Confusion matrix: 
 [[123  21]
 [ 19 125]]
From the confusion matrix above there are 123 true negatives and 21 false positives.
There are 19 false negatives and 125 true positives. 

ResNeXt50
Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  83.12 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 82.5945945945946
The best parameters are:  {'logisticregression__C': 0.001} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  80.9 %
F1 score is:  0.81
Recall score is:  0.81
Precision sc

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  85.71 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 81.94594594594594
The best parameters are:  {'logisticregression__C': 0.0001} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  80.56 %
F1 score is:  0.81
Recall score is:  0.81
Precision score is:  0.81
Confusion matrix: 
 [[112  32]
 [ 24 120]]
From the confusion matrix above there are 112 true negatives and 32 false positives.
There are 24 false negatives and 120 true positives. 

EfficientNetB4
Train:  (921, 175616) Validation: (231, 175616) Test: (288, 175616)
Validation score of EfficientNetB4 model:  85.28 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 82.27027027027027
The best parameters are:  {'logisticregression__C': 0.0005} 

Testing Accuracy on totally unseen data of EfficientNetB4 model:  80.21 %
F1 score is:  0.8
Recall score is:

...

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNet50 model:  87.45 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 82.97297297297297
The best parameters are:  {'logisticregression__C': 0.0005} 

Testing Accuracy on totally unseen data of ResNet50 model:  82.64 %
F1 score is:  0.83
Recall score is:  0.83
Precision score is:  0.83
Confusion matrix: 
 [[118  26]
 [ 24 120]]
From the confusion matrix above there are 118 true negatives and 26 false positives.
There are 24 false negatives and 120 true positives. 

ResNeXt50
Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  81.82 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 80.7027027027027
The best parameters are:  {'logisticregression__C': 0.001} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  80.9 %
F1 score is:  0.81
Recall score is:  0.81
Precision sc

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNet50 model:  85.71 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 84.1081081081081
The best parameters are:  {'logisticregression__C': 0.001} 

Testing Accuracy on totally unseen data of ResNet50 model:  82.99 %
F1 score is:  0.83
Recall score is:  0.83
Precision score is:  0.83
Confusion matrix: 
 [[119  25]
 [ 24 120]]
From the confusion matrix above there are 119 true negatives and 25 false positives.
There are 24 false negatives and 120 true positives. 

ResNeXt50
Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  81.82 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 83.2972972972973
The best parameters are:  {'logisticregression__C': 0.0001} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  81.94 %
F1 score is:  0.82
Recall score is:  0.82
Precision sc

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNet50 model:  80.95 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 85.56756756756756
The best parameters are:  {'logisticregression__C': 0.001} 

Testing Accuracy on totally unseen data of ResNet50 model:  84.03 %
F1 score is:  0.84
Recall score is:  0.84
Precision score is:  0.84
Confusion matrix: 
 [[119  25]
 [ 21 123]]
From the confusion matrix above there are 119 true negatives and 25 false positives.
There are 21 false negatives and 123 true positives. 

ResNeXt50
Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  83.98 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 82.91891891891892
The best parameters are:  {'logisticregression__C': 0.0001} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  79.51 %
F1 score is:  0.79
Recall score is:  0.8
Precision s

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNet50 model:  78.79 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 84.64864864864865
The best parameters are:  {'logisticregression__C': 0.001} 

Testing Accuracy on totally unseen data of ResNet50 model:  84.03 %
F1 score is:  0.84
Recall score is:  0.84
Precision score is:  0.84
Confusion matrix: 
 [[119  25]
 [ 21 123]]
From the confusion matrix above there are 119 true negatives and 25 false positives.
There are 21 false negatives and 123 true positives. 

ResNeXt50
Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  78.35 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 82.21621621621622
The best parameters are:  {'logisticregression__C': 0.0005} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  83.33 %
F1 score is:  0.83
Recall score is:  0.83
Precision 

Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNet50 model:  83.12 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 85.18918918918918
The best parameters are:  {'logisticregression__C': 0.0005} 

Testing Accuracy on totally unseen data of ResNet50 model:  84.72 %
F1 score is:  0.85
Recall score is:  0.85
Precision score is:  0.85
Confusion matrix: 
 [[122  22]
 [ 22 122]]
From the confusion matrix above there are 122 true negatives and 22 false positives.
There are 22 false negatives and 122 true positives. 

ResNeXt50
Train:  (921, 200704) Validation: (231, 200704) Test: (288, 200704)
Validation score of ResNeXt50 model:  83.12 %
Grid search has completed. 
 Mean cross-validated score of the best estimator is: 81.94594594594594
The best parameters are:  {'logisticregression__C': 0.0001} 

Testing Accuracy on totally unseen data of ResNeXt50 model:  83.33 %
F1 score is:  0.83
Recall score is:  0.83
Precision

## Note
- The results are contained in the files: 
*'FE_and_AvgEnsNet7__10_fold_crossval_Seed_42_and_Seed_7_Results.csv'* ,          *'FE_and_AvgEnsNet7__10_fold_crossval_Seed_42_Preds.csv'* and *'FE_and_AvgEnsNet7__10_fold_crossval_Seed_7_Preds.csv'*.